<a href="https://colab.research.google.com/github/tiaaburton/Milestone-II-Machine-Learning-Project/blob/main/360Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uncomment below to install dependencies

In [2]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pyarrow
import os 
from datetime import datetime
from tqdm import tqdm

import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/compact-scene-317315-56e479e9e148.json'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Exploring the Sample Dataset

To use the dataset for the project, we need to do feature engineering. This allows us to get a better model, but firstly we will look at the columns of a sample dataset that is only consisting of data from one date.

**Write a section about need to have a google storge account and how that's related to cost: https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas ; quotas and limits: https://cloud.google.com/bigquery/quotas

** Need to remake this section with PySpark with https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

## 1 Year Dataset

Given the time constraints by the dataset. We will have only 1 year of data potentially available. Using pyarrow and pandas, we will extract data and manipulate the data for continued use and understanding of the dataset.

In [8]:
sample_dataset = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/sample_file.csv')

In [13]:
def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    return ast.literal_eval(d)

def list_of_dicts(ld, field):
    '''
    Create a mapping of the tuples formed after 
    converting json strings of list to a python list   
    '''
    try:
      result = []
      for d in ast.literal_eval(ld):
        if d == None:
          continue
        else:
          result.append(d[field])
      return result
    except (TypeError, ValueError):
      return only_dict(ld)[field]
    else:
      return list_of_dicts(ld, field)

def data_from_json(data, field):
  """Using the rows from the column and the field wanting to be selected
  the function finds the best method to parse the json code and return
  the nested value."""
  from json import JSONDecodeError

  try:
    json_data = json.dumps(data)
    json_field = json.loads(json_data.replace('\'', '\"')[1:-1])[field]
    return json_field
  except TypeError:
    return only_dict(data)[field]
  except JSONDecodeError:
    json_data = json.dumps(data)
    return list_of_dicts(json.loads(json_data), field)
  else:
    return None

def get_selected_columns(filename='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt'):
  """
  filename: """
  columns = []
  with open(filename, 'r') as f:
    for ln in f.readlines():
      col = (ln.strip().split('\xa0\xa0')[1])
      groups = re.search(r"'(\w.+)'", col)
      columns.append(groups.group(1))
  return columns

def preprocessing(sample, columns:list):
  drop_cols = set()

  for col in tqdm(columns):
    try:
      splits = col.split('.')
      colName = ".".join(splits)
      if colName in sample.columns:
        continue
      elif len(splits) == 2:
        sample[colName] = sample[splits[0]].apply(lambda x: data_from_json(x, splits[1]))
      elif len(splits) == 3:
        prefix = ".".join(splits[:2])
        if prefix in sample.columns:
          sample[colName] = sample[prefix].apply(lambda x: data_from_json(x, splits[2]))
        else:
          sample[prefix] = sample[splits[0]].apply(lambda x: data_from_json(x, splits[1]))
          sample[colName] = sample[prefix].apply(lambda x: data_from_json(x, splits[2]))
        drop_cols.add(splits[0])
        drop_cols.add(prefix)
    except:
      continue
  
  return sample, drop_cols

In [16]:
columns = get_selected_columns()
print(columns)

['date', 'totals', 'fullVisitorId', 'clientId', 'totals.visits', 'totals.hits', 'totals.pageviews', 'totals.timeOnSite', 'totals.bounces', 'totals.transactions', 'totals.newVisits', 'totals.screenviews', 'totals.uniqueScreenviews', 'totals.timeOnScreen', 'totals.totalTransactionRevenue', 'hits.type', 'hits', 'channelGrouping', 'socialEngagementType', 'geoNetwork.country', 'trafficSource.source', 'trafficSource.medium', 'trafficSource.isTrueDirect', 'device.browser', 'device.operatingSystem', 'device.deviceCategory', 'hits.hour', 'hits.minute', 'hits.transaction', 'hits.item', 'hits.dataSource', 'hits.transaction.transactionRevenue', 'hits.item.productName', 'hits.item.productCategory', 'hits.item.itemRevenue', 'hits.appInfo.screenDepth', 'hits.eCommerceAction.action_type']


In [177]:
processed_data, drop_cols = preprocessing(sample_dataset, columns)

100%|██████████| 37/37 [42:06<00:00, 68.30s/it] 


In [243]:
remaining = list(set(columns).difference(set(processed_data.columns)))
for col in remaining:
  print(col)

hits.item.productCategory
hits.item.itemRevenue
hits.transaction.transactionRevenue
hits.eCommerceAction.action_type
clientId
hits.item.productName
hits.appInfo.screenDepth


In [222]:
processed_data.to_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv')

In [223]:
processed_data.iloc[0]

date                                                                       20160801
geoNetwork                        {'continent': 'Americas', 'subContinent': 'Nor...
channelGrouping                                                              Social
fullVisitorId                                                   7305625498291809599
socialEngagementType                                           Not Socially Engaged
trafficSource                     {'referralPath': '/yt/about/', 'campaign': '(n...
device                            {'browser': 'Internet Explorer', 'browserVersi...
totals                            {'visits': 1, 'hits': 4, 'pageviews': 2, 'time...
hits                              [{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...
hits.type                                                [PAGE, EVENT, PAGE, EVENT]
hits.hour                                                          [15, 15, 15, 15]
hits.minute                                                        [33, 34, 

## Examine Saved File 

In [21]:
transformation = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv', index_col=0)

In [39]:
remaining = list(set(columns).difference(set(transformation.columns)))
remaining

['clientId']

In [44]:
transformation.shape

(35725, 41)

In [46]:
transformation.tail()

,date,geoNetwork,channelGrouping,fullVisitorId,socialEngagementType,trafficSource,device,totals,hits,hits.type,...,geoNetwork.country,device.operatingSystem,device.deviceCategory,totals.pageviews,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.itemRevenue,hits.item.productCategory
35720,20161130,"{'continent': 'Americas', 'subContinent': 'Nor...",Social,5417408500825735224,Not Socially Engaged,"{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",['PAGE'],...,United States,Windows,desktop,1.0,[0],[0],[None],[None],[None],[None]
35721,20161130,"{'continent': 'Asia', 'subContinent': 'Southea...",Organic Search,8847411995415974974,Not Socially Engaged,"{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",['PAGE'],...,Indonesia,Android,mobile,1.0,[0],[0],[None],[None],[None],[None]
35722,20161130,"{'continent': 'Americas', 'subContinent': 'Nor...",Organic Search,163906276211779928,Not Socially Engaged,"{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",['PAGE'],...,United States,Windows,desktop,1.0,[0],[0],[None],[None],[None],[None]
35723,20161130,"{'continent': 'Americas', 'subContinent': 'Nor...",Referral,207342271544307763,Not Socially Engaged,"{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 14, 'minu...","['PAGE', 'PAGE']",...,United States,Chrome OS,desktop,2.0,"[0, 0]","[0, 0]","[None, None]","[None, None]","[None, None]","[None, None]"
35724,20161130,"{'continent': 'Americas', 'subContinent': 'Nor...",Organic Search,7263171742001601494,Not Socially Engaged,"{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...","['PAGE', 'PAGE']",...,United States,iOS,mobile,2.0,"[0, 0]","[0, 0]","[None, None]","[None, None]","[None, None]","[None, None]"


## Method in PySpark

In [47]:
! pip install pyspark

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 44.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0a7db4c631183b02b7405d0f5a1d09f158affce151c592c92fedc4b708ac6dc1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [48]:
import pyspark